# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
from collections import Counter
# import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    
    uniqueWords = Counter(text)
    vocab_to_int = {val:idx for idx, val in enumerate(uniqueWords)}
    int_to_vocab = {idx:val for idx, val in enumerate(uniqueWords)}

    return vocab_to_int, int_to_vocab

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    token_dict = { "." : "<<period>>",
                   "," : "<<comma>>",
                   "\"" : "<<quotation>>",
                   ";" : "<<semicolon>>",
                   "!" : "<<exclamation>>",
                   "?" : "<<question>>",
                   "(" : "<<l_parenthesis>>",
                   ")" : "<<r_parenthesis>>",
                   "--" : "<<dash>>",
                   "\n" : "<<return>>"
                 }
    return token_dict

## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
# import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

Notice that the last target value in the last batch is the first input value of the first batch. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [7]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    words_per_batch = batch_size*seq_length
    possible_num_batches = len(int_text)//words_per_batch

    for idx in range(0, possible_num_batches):
        x = int_text[words_per_batch*idx:(idx+1)*words_per_batch]

        if(((idx+1)*words_per_batch)+1<len(int_text)):
            y = int_text[(words_per_batch*idx)+1:((idx+1)*words_per_batch)+1]
        else:
            y = int_text[(words_per_batch*idx)+1:((idx+1)*words_per_batch)]
            y.append(int_text[0])

        x = np.array(x).reshape((batch_size, seq_length))
        y = np.array(y).reshape((batch_size, seq_length))
        yield x, y
    
    #check leftover data and return one last partial batch
    leftover_data = int_text[possible_num_batches*words_per_batch:]
    possible_num_sequences = len(leftover_data)//seq_length
    x = []
    y = []
    for i in range(0, possible_num_sequences):
        x.append(leftover_data[i*seq_length:(i+1)*seq_length])

        if(((i+1)*seq_length)+1<len(leftover_data)):
            temp = leftover_data[(i*seq_length)+1:((i+1)*seq_length)+1]
        else:
            temp = leftover_data[(i*seq_length)+1:(i+1)*seq_length]
            temp.append(int_text[0])
        y.append(temp)

    x = np.array(x).reshape((-1, seq_length))
    y = np.array(y).reshape((-1, seq_length))
    yield x, y

## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.

In [8]:
# Number of Epochs
num_epochs = 150
# Batch Size
batch_size = 500
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 50
# Sequence Length
seq_length = 15
# Learning Rate
learning_rate = 0.01

In [9]:
from tensorflow.keras import layers, Model
import tensorflow as tf

inputLayer = layers.Input((None,))

embeddingLayer = layers.Embedding(len(int_to_vocab), embed_dim)(inputLayer)

lstmLayer1 = layers.LSTM(rnn_size, return_sequences=True)(embeddingLayer)
lstmLayer2 = layers.LSTM(rnn_size, return_sequences=True)(lstmLayer1)

denseLayer = layers.Dense(len(int_to_vocab), activation=None)(lstmLayer2)

tv_script_model = Model(inputs=inputLayer, outputs=denseLayer, name="TV_SCRIPT_MODEL")
tv_script_model.summary()

Model: "TV_SCRIPT_MODEL"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 50)          339000    
_________________________________________________________________
lstm (LSTM)                  (None, None, 256)         314368    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 256)         525312    
_________________________________________________________________
dense (Dense)                (None, None, 6780)        1742460   
Total params: 2,921,140
Trainable params: 2,921,140
Non-trainable params: 0
_________________________________________________________________


In [10]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [ ]:
for e in range(num_epochs):

    for x, y in get_batches(int_text, batch_size=batch_size, seq_length=seq_length):

        with tf.GradientTape() as tape:
            logits = tv_script_model(x, training=True)
                
            y_one_hot = tf.one_hot(y, len(int_to_vocab))
            # print(logits.get_shape(), y_one_hot.get_shape())
            y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
            
            # Softmax cross entropy loss
            loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
            loss = tf.reduce_mean(loss)
        grads = tape.gradient(loss, tv_script_model.trainable_variables)
        optimizer.apply_gradients(zip(grads, tv_script_model.trainable_variables))
    if((e+1)%5==0):
        print ('Epoch {} finished with {} loss'.format(e+1, loss))
    


In [12]:
string = "moe_szyslak:".lower()
stringEnc = [vocab_to_int[i] for i in string.split()]
temp = tv_script_model(np.array(stringEnc).reshape(1,len(stringEnc),1))

word_index_to_append = np.argmax(temp[-1][-1].numpy())

for i in range(500):
    stringEnc.append(word_index_to_append)

    if(len(stringEnc)<seq_length):
        temp = tv_script_model(np.array(stringEnc).reshape(1, len(stringEnc), 1))
    else:
        temp = tv_script_model(np.array(stringEnc[-seq_length:]).reshape(1, seq_length, 1))

    predicted_ids = tf.random.categorical(temp[-1], num_samples=1)
    word_index_to_append = tf.squeeze(predicted_ids, axis=-1).numpy()[-1]

reversetoken = {val:key for key, val in token_dict.items()}

for i in stringEnc:
    token =  int_to_vocab[i]
    try:
        print(reversetoken[token], end=" ")
    except:
        print(token, end=" ")

moe_szyslak: ( friendly ) when you gonna pull the big frosty dollars ? 
 man: all true i'm telling the original money ) come on , tell you . . . 
 moe_szyslak: ah , this place has to make her right , but i should be english an heatherton . i'm trying . . . 
 moe_szyslak: okay , i only get put bums . we love you make that world of me . 
 moe_szyslak: ya know you come myself . ( box phone ) ah , where i gotta drink to take it too . 
 harv: when what my bad won't do it tonight , still back off from pretend at the drop-off , things must be get a name . 
 sideshow_bob: are it . i got it to drink , i gotta be eatin' " he can get me . 
 stillwater: ( considers business . . . two that in those and and slit outta you . 
 moe_szyslak: witty . 
 homer_simpson: yeah , you're okay i should always be warned . i love you told you , first . 
 
 
 moe_szyslak: the day you know just pair , i'm not treat everything . 
 homer_simpson: but they should take out of great . 
 seymour_skinner: you're the boy w